In [95]:
# Imports:
import numpy as np
import matplotlib.pyplot as plt

 

Exercise 1

In [146]:
# This is the function from the lab:
def f(x):
    return 1/(0.01+(x-0.3)**2)+1/(0.04+(x-0.9)**2)-6

In [37]:
# This is my implementation of the Equidistant Numerical Integration.
# Mainly from the rewritten from the lab exercise and ChatGPT
# Expenations. I have chosen to do a step zero, so I can have the simple
# formula in the for cycle.
def eq_integral(a, b, w, f, N):
    M = len(w)
    h = (b-a)/N

    result = 0
    
    for i in range(N):
        x_l = a + i * h
        x_r = a + (i+1) * h

        inner_h = (x_r - x_l)/(M-1)
        
        # Step 0:
        result += w[0]*f(x_l)

        for j in range(1,M):
           
           x = x_l+j*inner_h
           result += w[j]*f(x)

    result *= h
    
    return result


In [2]:
Q = 29.858325395498671;

In [193]:
# These are the absolute error values for different M's.
# I fixed the N to 50 for better results.

# First 
N = 50


p = 2
w = [1/2, 1/2]
a = 0
b = 1

result = eq_integral(a,b,w,f,N)
diff = abs(Q - result)

print("Result for M = 2: ")
print(result)
print("Diff: " + str(diff))
print("")

# Second:
p = 4
w = [1/6, 2/3, 1/6]

result = eq_integral(a,b,w,f,N)
diff = Q - result

print("Result for M = 3: ")
print(result)
print("Diff: " + str(diff))
print("")

# Third:
p = 4
w = [1/8, 3/8, 3/8, 1/8]

result = eq_integral(a,b,w,f,N)
diff = abs(Q - result)

print("Result for M = 4: ")
print(result)
print("Diff: " + str(diff))
print("")

# Fourth:
p = 6
w = [7/90, 16/45, 4/30, 16/45, 7/90]

result = eq_integral(a,b,w,f,N)
diff = abs(Q - result)

print("Result for M = 5: ")
print(result)
print("Diff: " + str(diff))
print("")


Result for M = 2: 
29.85338776701903
Diff: 0.004937628479641631

Result for M = 3: 
29.85832570840843
Diff: -3.1290975854858516e-07

Result for M = 4: 
29.858325534478706
Diff: 1.3898003459189567e-07

Result for M = 5: 
29.8583253954065
Diff: 9.21716036828002e-11



Adaptive Integral:

In [83]:
# This function is for the rule, called in the recursive part adaptive_integral():
def re(f, a, b,w):
    h = (b-a)/(len(w)-1)

    # Step zero:
    result = w[0]*f(a)

    for i in range(1,len(w)):
        result += f(a+i*h)*w[i]
    
    #scaling:
    result *= (b-a)
    return result 

# This is the recursive part, where I create two
# subintervals (Q1 = left, Q2 = right) if the difference from
# Q is too big regarding our tolerance.
def adaptive_integral(a, b, f, w,tol,p,interval = 0):
    
    Q = re(f, a, b, w)

    m = (a + b) / 2

    # Q1 and Q2
    left = re(f,a,m,w)
    right = re(f,m,b,w)

    # Equation from the Lab1 notes the second step.
    err = abs(left + right - Q)/((2**p)-1)
    
    
    if tol < err:
        
        # This is the fourth step of the algorithm.
        left,left_int = adaptive_integral(a, m, f, w, tol/2,p,interval)
        right,right_int = adaptive_integral(m, b, f, w, tol/2,p,interval)
        
        return left + right,left_int+right_int
    else:
        # If the error is smaller than the tolerance, than we return, and increment the intervals.
        return left + right, interval+1






In [194]:
# This part is the same as the previous for equidistant.
# I have fixed the tolerance at 1e-6, for different M's.

# First
tol = 1e-6


p = 2
w = [1/2, 1/2]
a = 0
b = 1

result,_ = adaptive_integral(a,b,f,w,tol,p)
diff = abs(Q - result)

print("Result for M = 2: ")
print(result)
print("Diff: " + str(diff))
print("")

# Second:
p = 4
w = [1/6, 2/3, 1/6]

result,_ = adaptive_integral(a,b,f,w,tol,p)
diff = abs(Q - result)

print("Result for M = 3: ")
print(result)
print("Diff: " + str(diff))
print("")

# Third:
p = 4
w = [1/8, 3/8, 3/8, 1/8]

result,_ = adaptive_integral(a,b,f,w,tol,p)
diff = abs(Q - result)

print("Result for M = 4: ")
print(result)
print("Diff: " + str(diff))
print("")

# Fourth:
p = 6
w = [7/90, 16/45, 4/30, 16/45, 7/90]

result,_ = adaptive_integral(a,b,f,w,tol,p)
diff = abs(Q - result)

print("Result for M = 5: ")
print(result)
print("Diff: " + str(diff))
print("")


Result for M = 2: 
29.85832560502142
Diff: 2.0952274937258153e-07

Result for M = 3: 
29.85832542436354
Diff: 2.8864867829270224e-08

Result for M = 4: 
29.858325435719618
Diff: 4.022094657329944e-08

Result for M = 5: 
29.85832554980515
Diff: 1.543064804820915e-07



Exercise 2:

In [93]:
# I have chose to calcuate with M = 3 with the given weights.
# Here I calculate the absolute error for all tolerances.
p = 4
w = [1/6, 2/3, 1/6]
a = 0
b = 1

tolerances = [1., 0.1, 0.01, 1e-3, 1e-4, 1e-5, 1e-6, 1e-7, 1e-8]
for _,tol in enumerate(tolerances):
    result, interval = adaptive_integral(a,b,f,w,tol,p)
    
    diff = abs(Q - result)
    print("Result for tolerance "+str(tol)+": "+str(result)+ " | Diff: " + str(diff)+" | Interval: "+ str(interval))
    

Result for tolerance 1.0: 29.070213115105005 | Diff: 0.7881122803936655 | Interval: 2
Result for tolerance 0.1: 29.926700276433547 | Diff: 0.06837488093487565 | Interval: 4
Result for tolerance 0.01: 29.83903353312538 | Diff: 0.01929186237329006 | Interval: 7
Result for tolerance 0.001: 29.85835729425238 | Diff: 3.189875370779305e-05 | Interval: 16
Result for tolerance 0.0001: 29.858324437224447 | Diff: 9.582742244163e-07 | Interval: 24
Result for tolerance 1e-05: 29.85832684503596 | Diff: 1.4495372901990322e-06 | Interval: 45
Result for tolerance 1e-06: 29.85832542436354 | Diff: 2.8864867829270224e-08 | Interval: 81
Result for tolerance 1e-07: 29.85832539651661 | Diff: 1.0179377341046347e-09 | Interval: 150
Result for tolerance 1e-08: 29.85832539593435 | Diff: 4.3567993657234183e-10 | Interval: 266


In [137]:
# Here I just searched for N values to find the correct output
# for each tolarance with trial and error.
N = 210

p = 4
w = [1/6, 2/3, 1/6]
result = eq_integral(a,b,w,f,N)
diff = abs(Q - result)

print("Result for M = 3: ")
print(result)
print("Diff: " + str(diff))
print("")

Result for M = 3: 
29.85832539649866
Diff: 9.999894245993346e-10



In [138]:
# These are the results for N's:
N_for_tolarences = [5, 9,10,19,25,70,119,210]
# These were the values for toalrence level:
# tol = 1.0       | N = 5
# tol = 0.1       | N = 9
# tol = 0.01      | N = 10
# tol = 0.001     | N = 19
# tol = 1e-4/1e-5 | N = 25
# tol = 1e-6      | N = 70
# tol = 1e-7      | N = 119
# tol = 1e-8      | N = 210



Exercise 3:

In [195]:
# I have written this part and determined the weights and the node with the help of chatgpt and from lecture:
# (https://math.libretexts.org/Workbench/Numerical_Methods_with_Applications_(Kaw)/7%3A_Integration/7.05%3A_Gauss_Quadrature_Rule_of_Integration)

def Gaussian_quadrature(f,a,b,w,nodes):
    # Calculating according the referenced lecture:
    x1 = (b-a)/2*nodes[0] + (b+a)/2
    x2 = (b-a)/2*nodes[1] + (b+a)/2

    # Rule applied:
    integral = w[0]*f(x1)+w[1]*f(x2)
    
    # Scaling of the result
    integral *= (b - a) / 2

    return integral

def divide_Gaussian(f,a,b,w,nodes,N):
    h = (b-a)/(N-1)

    # step zero:
    result = Gaussian_quadrature(f,a,a+h,w,nodes)
    

    for i in range(1,N-1):
        result += Gaussian_quadrature(f,a+(h*i),a+h*(i+1),w,nodes)

    return result


# N is the number of subintervals:
N = 15
a = 0
b = 1

weights = [1, 1]
nodes = np.array([-1/np.sqrt(3), 1/np.sqrt(3)])

result = divide_Gaussian(f, a, b,weights,nodes,N)
diff = abs(Q - result)

print("Result for 2-point Gaussian: ")
print(result)
print("Diff: " + str(diff))
print("")






Result for 2-point Gaussian: 
29.858996929092402
Diff: 0.0006715335937315103



Exercises 4:

In [196]:
# These are just some functions to help me order the results, by passing
# absolute difference values into this finction and it gives back the ordered version of it.
def sortFn(dict):
    return dict['value']    

def order(a,e,G):

    values = [{
    'name': 'Adaptive',
    'value': a
    }, {
        'name': 'Equidistant',
        'value': e
    }, {
        'name': '2-point Gaussian',
        'value': G
    }]

    values = sorted(values, key=sortFn)
    return values



In [197]:
# parameters I have used:
p = 4
w = [1/6, 2/3, 1/6]
a = 0
b = 1


# previously calculated for equidistant:
N_for_tolarences = [5, 9,10,19,25,70,119,210]
tolerances = [1., 0.1, 0.01, 1e-4, 1e-5, 1e-6, 1e-7, 1e-8]

for i,tol in enumerate(tolerances):
    N = N_for_tolarences[i]

    result_a, interval = adaptive_integral(a,b,f,w,tol,p)
    result_G = divide_Gaussian(f, a, b,weights,nodes,N)
    result_e = eq_integral(a,b,w,f,N)

    diff_a = abs(Q - result_a)
    diff_G = abs(Q - result_G)
    diff_e = abs(Q - result_e)

    sorted_diff = order(diff_a,diff_e,diff_G)
    
    print("Result of adaptive method for tolarance: "+str(tol)+": "+str(result_a)+ " | Diff: " + str(diff_a)+" | Interval: "+ str(interval))
    print("Result Gaussian for interval " +str(N)+": "+str(result_G)+ " | Diff: " + str(diff_G)+" | Interval: "+ str(N))
    print("Result of equidistant for interval " +str(N)+": "+str(result_e)+ " | Diff: " + str(diff_e)+" | Interval: "+ str(N))
    print("The order of methods relative to the calculated value Q: "+ sorted_diff[0]['name'] +" < "+ sorted_diff[1]['name'] + " < " + sorted_diff[2]['name'])
    print()
    

# In most cases, adaptive with 3 points performed the best.
# But all of the above methods were close to the original Q.

Result of adaptive method for tolarance: 1.0: 29.070213115105005 | Diff: 0.7881122803936655 | Interval: 2
Result Gaussian for interval 5: 30.507493436780127 | Diff: 0.649168041281456 | Interval: 5
Result of equidistant for interval 5: 30.904870193721656 | Diff: 1.0465447982229854 | Interval: 5
The order of methods relative to the calculated value Q: 2-point Gaussian < Adaptive < Equidistant

Result of adaptive method for tolarance: 0.1: 29.926700276433547 | Diff: 0.06837488093487565 | Interval: 4
Result Gaussian for interval 9: 29.777082694523855 | Diff: 0.08124270097481556 | Interval: 9
Result of equidistant for interval 9: 29.88057864655919 | Diff: 0.02225325106051912 | Interval: 9
The order of methods relative to the calculated value Q: Equidistant < Adaptive < 2-point Gaussian

Result of adaptive method for tolarance: 0.01: 29.83903353312538 | Diff: 0.01929186237329006 | Interval: 7
Result Gaussian for interval 10: 29.84237781012286 | Diff: 0.015947585375812423 | Interval: 10
Resul